### graph data modeling

learn
    what data modeling is?
    
    why model?
    - design graph that is:
      - answer the key use cases fpr the application
      - provide best Cypher query performance
    - How to create a model
    - 1. understand the domain and define specific use cases for application 
    - 2. develop the initial graph data model:
      - a. model the node (entities)
      - b. model relationships between the nodes
    - 3. Test the use cases agains the initial model
    - 4. Create the grap (instance model) against the use case
    - 5. Test the use cases, including performance against the graph
    - 6. refactor (improve) the graph data model due to a change in the key use cases or for performance
    - 7. implement refactoring and retest.

    As graphs scale - refactoring will typically need to accomadte lareger size

### the domain

# understand the domain of your use case
    * understand stakeholder and developers
      * do they understand the processes/app?
      * write app details in use
      * identify the users of the application (people,systems)
      * agree upon use cases for the application
      * RANK the importance of use cases

# use cases for the movie graph used in this course
    * what people acted in a movie?
    * what person directed a movie?
    * what movies did a person act in?
    * how many users rated a movie?
    * who was the youngest person to act in a movie?
    * what role did someone play in a movie?
    * what was the highest rated movie in a year 'x' according to imdb?
    * what drama movies did actor y act in?
    * what users gave movie m a 5.0 review?

# differentiate 

### Models in grpah

# data model
    labels, relationships, properties of graph
    no exact data
    relationships connect the nodes to each other.
    Nodes have properties
    agree how labels/properties/relationships assigned

        Best practice:
            - Label
              - single identifier that begins with a capital letter - can be camel cased : Person, Movie, GitHubRepo
            - Relationship type:
              - single identifier that is in ALL_CAPS and underscore separated 
              - ex. FARTED_ON
            - Property key:
              - single identifier for a node or identifier; begins with lower-case letter, can be camel-cased. 
                - ex. firstName
                - does not need to be unique!

# instance model
instance 


### modeling Nodes

# define labels

    Start with Dominant entities in use cases
        - what ingredients are used in a recipe?
        - who is married to which person?
    - labels determine how nodes are specified

    Nodes:
      - node properties do the following:
        - identify a node
        - answer specific details of the use case of application 
        - return property values



  # this creates the following nodes - but no relationships : yet

        MATCH (n) DETACH DELETE n;

        MERGE (:Movie {title: 'Apollo 13', tmdbId: 568, released: '1995-06-30', imdbRating: 7.6, genres: ['Drama', 'Adventure', 'IMAX']})
        MERGE (:Person {name: 'Tom Hanks', tmdbId: 31, born: '1956-07-09'})
        MERGE (:Person {name: 'Meg Ryan', tmdbId: 5344, born: '1961-11-19'})
        MERGE (:Person {name: 'Danny DeVito', tmdbId: 518, born: '1944-11-17'})
        MERGE (:Person {name: 'Jack Nicholson', tmdbId: 514, born: '1937-04-22'})
        MERGE (:Movie {title: 'Sleepless in Seattle', tmdbId: 858, released: '1993-06-25', imdbRating: 6.8, genres: ['Comedy', 'Drama', 'Romance']})
        MERGE (:Movie {title: 'Hoffa', tmdbId: 10410, released: '1992-12-25', imdbRating: 6.6, genres: ['Crime', 'Drama']})

  # this will allow you to input multiple users at the same time

      UNWIND [{userId: 534, name: 'Sandy Jones'}, {userId: 105, name: 'Clinton Spencer'}] AS  # create JSON element of items in a list!
      MERGE (u:User {userId: userData.userId, name: userData.name}) # merge them based on the keys



### Modeling relationships in your graphs



Connections are the <u>verbs</u> in your use cases:
    - choosing good names is super important
      - direction is either implied by l - t - r
      - or done in the creation
  
  # FAN OUT
  - Linked nodes
    - who has the last name scott?
  - This leads to VERY dense nodes 

# Enriching a relationship
    - give properties to an actual relationship!
      - use case asks specific question on how two nodes are related to each other
      - properties can be added to relationship to further describe it
      - specific for ONLY these 2 nodes!

# This starts creating the movie graph 
        MATCH (apollo:Movie {title: 'Apollo 13'})
        MATCH (tom:Person {name: 'Tom Hanks'})
        MATCH (meg:Person {name: 'Meg Ryan'})
        MATCH (danny:Person {name: 'Danny DeVito'})
        MATCH (sleep:Movie {title: 'Sleepless in Seattle'})
        MATCH (hoffa:Movie {title: 'Hoffa'})
        MATCH (jack:Person {name: 'Jack Nicholson'})

        // create the relationships between nodes
        MERGE (tom)-[:ACTED_IN {role: 'Jim Lovell'}]->(apollo)
        MERGE (tom)-[:ACTED_IN {role: 'Sam Baldwin'}]->(sleep)
        MERGE (meg)-[:ACTED_IN {role: 'Annie Reed'}]->(sleep)
        MERGE (danny)-[:ACTED_IN {role: 'Bobby Ciaro'}]->(hoffa)
        MERGE (danny)-[:DIRECTED]->(hoffa)
        MERGE (jack)-[:ACTED_IN {role: 'Jimmy Hoffa'}]->(hoffa)

# get aggregations
        this gets avg review by reviewer amd # of reviews
        MATCH (u:User) -[r:RATED]->(m:Movie)  RETURN u.name, count(*) as `num_ratings`, avg(r.rating) as `avg_rating`

# get this for every movie
        MATCH (u:User) -[r:RATED]->(m:Movie)   RETURN u.name, count(*) as `num_ratings`, round(avg(r.rating),2)  as `avg_rating`, m.title order by u.name, m.title



### Testing the model

# test against instance model
    design with
        - arrows app
        - whiteboard
        - paper
- write out the results that you expect from your whiteboarding and other work : then stress test it against the actual results
- this will help you know what data you want add into your use cases
  - graph may need to be refactored

Scalability the biggest way to test functionality
- how will qu`ery run with millions of nodes and relationships?
  
# example
# return the actors in sleepless in seattle
    match (p:Person) -[:ACTED_IN]->(m:Movie) where lower(m.title) =~ '^sleepl.*' return p.name

# count distinct people nodes
# this is a good template for counting distinct relationships in your graph!
    match (p:Person) return count(p)

# return if item IN a list
    MATCH (p:Person)-[:ACTED_IN]-(m:Movie)
    WHERE p.name = 'Tom Hanks' AND
    'Drama' IN m.genres
    RETURN m.title AS Movie

### Refactoring

# why need to refactor?
    - change the data model or graph
      - nodes labeled
      - relationships
      - graph does not answer all use cases
      - new use case has come up we must account for
      - Cypher for use case does not perform optimally when scaled
    - 1. design new data model
    - 2. cypher to transform existing graph to new model
    - 3. retest use cases

### Labels in the graph

# node labels:
    - anchor in query 
      - subset of 1+ nodes from which to start a query 
      - reduce size of graph that is touched by query
      - if there is also a label 
        - MATCH (n:US) return n # returns all us nodes which happen to be all person nodes
        - you cannot parametrize the label, so keeping it a parameter is more flexible
        - strong use case of multiple labels in node - do so/.
        - limit labels for node to 4 or less
          - and if A FEATURE FOF A NODE WILL SUFFICE - do not MAKE the item a label
  
  # possible use case for 2d label

    MATCH (p:Person)-[:ACTED_IN] -()
    WHERE p.born < 1950 
    RETURN p.name

    this does:
        - a node by label scan to return all people nodes
        - tests the born property and filters
        - then makes sure there is acted in relationship
        - then returns the name

    PROPERTY     
        MATCH (p:Person)-[:ACTED_IN] -()
        WHERE p.born < 1950 
        RETURN p.name

CACHED INSTANCES CAN BNOT BE COMPARED
but you can see the rows of returned at each step

Can change data model to have an actor label like so

    match (p:Person) # matches all person
    WHERE exists((p)-[ACTED_IN]-()) # with acted in relationship
    SET p.Actor # add actor label to the node

### Example

        # limit based on a filter in the node
        PROFILE MATCH (p:Person)-[:ACTED_IN]-()
        WHERE p.born < '1950'
        RETURN p.name

        THIS SHOWS THE qUERY RESULTS

        BUT if you add the new label of actor
            MATCH (p:Person)
            WHERE exists ((p)-[:ACTED_IN]-())
            SET p:Actor

        
        You can hen limit the amt of data queried 
            PROFILE MATCH (p:Actor)-[:ACTED_IN]-() this is ONLY the nodes that have a Actor label
            WHERE p.born < '1950'
            RETURN p.name

### Retesting after refactor

# what to do after refactor

    1. Identify use cases affected by refactor
    2. rewrite cypher to ensure correct results
    3. ensure all queries unaffected return same results as prior
    4. PROFILE queries to determine if refactoring helped

# ENSURE that actors are tagged accordingly
    MATCH (p:Actor) -[:ACTED_IN]->(n) return p.name

# see which movies actors acted in 
    MATCH (p:Actor) -[:ACTED_IN]->(m:Movie) return m.title

# Return only the youngest actor
    MATCH (q:Actor) -[:ACTED_IN]->()  return q.name, q.born  order by q.born desc limit 1

    OR

    MATCH (q:Actor) with max(q.born) as youngest MATCH (q:Actor) -[:ACTED_IN]->()  where q.born = youngest return q.name, q.born

    AND this gets the role as well

    MATCH (q:Actor) with max(q.born) as youngest MATCH (q:Actor) -[r:ACTED_IN]->(m:Movie)  where q.born = youngest return q.name, q.born, r.role

    and this is youngest actor in a drama

    MATCH (q:Actor) -[r:ACTED_IN]->(m:Movie)  WITH max(q.born) as youngest  
    MATCH (q:Actor) -[r:ACTED_IN]->(m:Movie) 
    WHERE lower('Drama') in [genre in m.genres|lower(genre)] 
    AND q.born = youngest 
    RETURN q.name, q.born, r.role

    # this sets the new label for Director

    MATCH (p:Person)-[:DIRECTED]->() set p:Director